# sketchnet4
Basic idea is to take sketchnet2 first layer, train two conv layers, then next two conv layers, etc

In [1]:
import keras
from keras.optimizers import rmsprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import backend as K

import numpy as np

import h5py

import tensorflow as tf

import utils
reload(utils)

%matplotlib inline

Using TensorFlow backend.


In [2]:
batch_size = 10

data_root = '../data/tu-berlin/sample50/'
train_data_dir = data_root + 'train'
validation_data_dir = data_root + 'test'



# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.25,
        zoom_range=0.25,
        rotation_range=35,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

batch, y = train_generator.next()
img_height = batch.shape[1]
img_width = batch.shape[2]
channels = batch.shape[3]
num_classes = len(y[0])
print img_height, img_width, channels, num_classes

Found 2650 images belonging to 50 classes.
Found 1350 images belonging to 50 classes.
256 256 1 50


In [3]:
model_name = 'sketchnet4_1'
%mkdir -p ../logs/{model_name}
%mkdir -p ../models/{model_name}

log_cb =\
    TensorBoard(log_dir='../logs/'+model_name+'/', 
                histogram_freq=0, 
                write_graph=False, write_images=False)
best_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/best.h5', 
                    monitor='val_acc', verbose=0, 
                    save_best_only=True, 
                    mode='auto', period=1)
latest_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/latest.h5', 
                    monitor='val_acc', verbose=0, 
                    period=1)

In [4]:
def build_model():
    model = Sequential()
    model.add(Convolution2D(64, (3, 3), 
                            padding='same',
                            activation='relu',
                            input_shape=(img_width, img_height, channels)))
    model.add(Convolution2D(64, (3, 3), 
                            padding='same',
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(128, (3, 3), 
                            padding='same', 
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # initialize optimizer
    opt = rmsprop(lr=0.0001)

#     model.compile(loss='categorical_crossentropy',
#                   optimizer=opt,
#                   metrics=['accuracy'])
    
    return model, opt

model, opt = build_model()

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 524288)            0         
__________

In [6]:
total_epochs = 0

In [7]:
nb_epoch = 1
K.set_value(opt.lr, 0.0001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

Epoch 1/1
265/265 [==============================] - 190s - loss: 4.0522 - acc: 0.0234 - val_loss: 3.7717 - val_acc: 0.0652


In [ ]:
nb_epoch = 5
K.set_value(opt.lr, 0.001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

Epoch 2/6
265/265 [==============================] - 187s - loss: 4.0205 - acc: 0.0200 - val_loss: 3.8907 - val_acc: 0.0422
Epoch 3/6
265/265 [==============================] - 205s - loss: 3.7893 - acc: 0.0566 - val_loss: 3.2446 - val_acc: 0.1659
Epoch 4/6
240/265 [==========================>...] - ETA: 14s - loss: 3.3692 - acc: 0.1450

In [ ]:
nb_epoch = 20
K.set_value(opt.lr, 0.001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

In [ ]:
metrics = model.evaluate_generator(train_generator, steps=265)
print 'train', model.metrics_names, metrics
metrics = model.evaluate_generator(validation_generator, steps=135)
print 'val', model.metrics_names, metrics

train ['loss', 'acc'] [2.264826936092017, 0.40490566837900088]
val ['loss', 'acc'] [2.3791824526256984, 0.44074074990219542]


In [5]:
# now pop the dense layer, add 2 conv layers + dense
# freeze bottom + tune,
# then train all

model, opt = build_model()
model.load_weights('../models/sketchnet4_1/latest.h5')

# pop 6 layers
for i in xrange(6):
    model.pop()
    
model.add(Convolution2D(128, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 256)       295168    
__________

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 256)       295168    
__________

In [17]:
model_name = 'sketchnet4_2'
%mkdir -p ../logs/{model_name}
%mkdir -p ../models/{model_name}

log_cb2 =\
    TensorBoard(log_dir='../logs/'+model_name+'/', 
                histogram_freq=0, 
                write_graph=False, write_images=False)
best_model_cb2 =\
    ModelCheckpoint('../models/'+model_name+'/best.h5', 
                    monitor='val_acc', verbose=0, 
                    save_best_only=True, 
                    mode='auto', period=1)
latest_model_cb2 =\
    ModelCheckpoint('../models/'+model_name+'/latest.h5', 
                    monitor='val_acc', verbose=0, 
                    period=1)

In [18]:
total_epochs = 0

In [19]:
# first tune:
for layer in model.layers[:4]:
    layer.trainable = False

nb_epoch = 1
K.set_value(opt.lr, 0.0001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb2, 
                   best_model_cb2,
                   latest_model_cb2]
)
total_epochs += nb_epoch

Epoch 1/1
265/265 [==============================] - 116s - loss: 3.8213 - acc: 0.0532 - val_loss: 3.4789 - val_acc: 0.1400


In [ ]:
nb_epoch = 5
K.set_value(opt.lr, 0.001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb2, 
                   best_model_cb2,
                   latest_model_cb2]
)
total_epochs += nb_epoch

Epoch 2/6
265/265 [==============================] - 130s - loss: 3.5782 - acc: 0.1075 - val_loss: 3.0185 - val_acc: 0.2415
Epoch 3/6
265/265 [==============================] - 130s - loss: 2.9958 - acc: 0.2177 - val_loss: 2.6279 - val_acc: 0.3193
Epoch 4/6
265/265 [==============================] - 121s - loss: 2.7282 - acc: 0.2796 - val_loss: 2.5986 - val_acc: 0.3163
Epoch 5/6
265/265 [==============================] - 130s - loss: 2.6335 - acc: 0.2974 - val_loss: 2.4777 - val_acc: 0.3941
Epoch 6/6
265/265 [==============================] - 120s - loss: 2.5480 - acc: 0.3147 - val_loss: 2.4126 - val_acc: 0.3904


In [ ]:
nb_epoch = 10
K.set_value(opt.lr, 0.0005)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb2, 
                   best_model_cb2,
                   latest_model_cb2]
)
total_epochs += nb_epoch

Epoch 7/16
265/265 [==============================] - 120s - loss: 2.1737 - acc: 0.4079 - val_loss: 2.3278 - val_acc: 0.4400
Epoch 12/16
265/265 [==============================] - 130s - loss: 2.1823 - acc: 0.4068 - val_loss: 2.3298 - val_acc: 0.4593
Epoch 13/16
265/265 [==============================] - 130s - loss: 2.1354 - acc: 0.4181 - val_loss: 2.2819 - val_acc: 0.4726
Epoch 14/16
265/265 [==============================] - 130s - loss: 2.1083 - acc: 0.4226 - val_loss: 2.1014 - val_acc: 0.4926
Epoch 15/16
265/265 [==============================] - 120s - loss: 2.0776 - acc: 0.4430 - val_loss: 2.3214 - val_acc: 0.4763
Epoch 16/16
265/265 [==============================] - 120s - loss: 2.0707 - acc: 0.4468 - val_loss: 2.2521 - val_acc: 0.4570


In [ ]:
# unfreeze network and train more
for layer in model.layers:
    layer.trainable = True
    
nb_epoch = 10
K.set_value(opt.lr, 0.0005)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb2, 
                   best_model_cb2,
                   latest_model_cb2]
)
total_epochs += nb_epoch

In [22]:
metrics = model.evaluate_generator(validation_generator, steps=135)
print 'val', model.metrics_names, metrics

val ['loss', 'acc'] [2.188649030526479, 0.48518519760281953]


In [ ]:
nb_epoch = 10
K.set_value(opt.lr, 0.00025)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb2, 
                   best_model_cb2,
                   latest_model_cb2]
)
total_epochs += nb_epoch

In [24]:
metrics = model.evaluate_generator(validation_generator, steps=135)
print 'val', model.metrics_names, metrics

val ['loss', 'acc'] [2.0498745988916469, 0.50296297437614867]


In [3]:
# load latest, add more convs

model = keras.models.load_model('../models/sketchnet4_2/latest.h5')
for i in xrange(5):
    model.pop()
    
model.add(Convolution2D(256, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

In [6]:
opt = rmsprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 256)       295168    
__________

In [12]:
model_name = 'sketchnet4_3'
%mkdir -p ../logs/{model_name}
%mkdir -p ../models/{model_name}

log_cb3 =\
    TensorBoard(log_dir='../logs/'+model_name+'/', 
                histogram_freq=0, 
                write_graph=False, write_images=False)
best_model_cb3 =\
    ModelCheckpoint('../models/'+model_name+'/best.h5', 
                    monitor='val_acc', verbose=0, 
                    save_best_only=True, 
                    mode='auto', period=1)
latest_model_cb3 =\
    ModelCheckpoint('../models/'+model_name+'/latest.h5', 
                    monitor='val_acc', verbose=0, 
                    period=1)    
lr_cb3 = \
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                  patience=3, min_lr=0.0001)
def sched_lr(e):
    if e == 0:
        return 0.0001
    elif e < 10:
        return 0.001
    elif e < 20:
        return 0.0005
    elif e < 30:
        return 0.00025
    else:
        return 0.0001
lr_sched_cb3 =\
    keras.callbacks.LearningRateScheduler(sched_lr)


In [13]:
total_epochs = 0

In [ ]:
nb_epoch = 25

for layer in model.layers[:7]:
    layer.trainable = False

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb3, 
                   best_model_cb3,
                   latest_model_cb3,
                   lr_sched_cb3]
)
total_epochs += nb_epoch

Epoch 1/25
265/265 [==============================] - 142s - loss: 2.7601 - acc: 0.2698 - val_loss: 2.3304 - val_acc: 0.3859
Epoch 2/25
265/265 [==============================] - 136s - loss: 3.2881 - acc: 0.1709 - val_loss: 2.4875 - val_acc: 0.3459
Epoch 3/25
265/265 [==============================] - 135s - loss: 2.7868 - acc: 0.2732 - val_loss: 2.3856 - val_acc: 0.3622
Epoch 4/25
265/265 [==============================] - 140s - loss: 2.4907 - acc: 0.3275 - val_loss: 2.1628 - val_acc: 0.4437
Epoch 5/25
265/265 [==============================] - 140s - loss: 2.3500 - acc: 0.3694 - val_loss: 1.8520 - val_acc: 0.4941
Epoch 6/25
265/265 [==============================] - 135s - loss: 2.2110 - acc: 0.4038 - val_loss: 2.0812 - val_acc: 0.4933
Epoch 7/25
265/265 [==============================] - 140s - loss: 2.1870 - acc: 0.4400 - val_loss: 1.9226 - val_acc: 0.5067
Epoch 8/25
265/265 [==============================] - 140s - loss: 2.0035 - acc: 0.4743 - val_loss: 1.9301 - val_acc: 0.5267


In [ ]:
nb_epoch = 25

for layer in model.layers[:7]:
    layer.trainable = True

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb3, 
                   best_model_cb3,
                   latest_model_cb3,
                   lr_sched_cb3]
)
total_epochs += nb_epoch

Epoch 26/50
265/265 [==============================] - 135s - loss: 1.1174 - acc: 0.6943 - val_loss: 1.5360 - val_acc: 0.6667
Epoch 27/50
265/265 [==============================] - 135s - loss: 1.1666 - acc: 0.6928 - val_loss: 1.6084 - val_acc: 0.6674
Epoch 28/50
265/265 [==============================] - 135s - loss: 1.1378 - acc: 0.6977 - val_loss: 1.5815 - val_acc: 0.6526
Epoch 29/50
265/265 [==============================] - 135s - loss: 1.1495 - acc: 0.6906 - val_loss: 1.4542 - val_acc: 0.6637
Epoch 30/50
265/265 [==============================] - 135s - loss: 1.1364 - acc: 0.6951 - val_loss: 1.5805 - val_acc: 0.6644
Epoch 31/50
265/265 [==============================] - 135s - loss: 1.0330 - acc: 0.7192 - val_loss: 1.6188 - val_acc: 0.6644
Epoch 32/50
265/265 [==============================] - 135s - loss: 1.0140 - acc: 0.7272 - val_loss: 1.7414 - val_acc: 0.6496
Epoch 33/50
265/265 [==============================] - 140s - loss: 1.0109 - acc: 0.7230 - val_loss: 1.5828 - val_acc:

KeyboardInterrupt: 

In [4]:
# load latest, add more convs
# model = keras.models.load_model('../models/sketchnet4_3/best.h5')
model.load_weights('../models/sketchnet4_3/best.h5')
for i in xrange(5):
    model.pop()
    
for i, layer in enumerate(model.layers):
    layer.name = 'layer'+str(i)
    
model.add(Convolution2D(512, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer0 (Conv2D)              (None, 256, 256, 64)      640       
_________________________________________________________________
layer1 (Conv2D)              (None, 256, 256, 64)      36928     
_________________________________________________________________
layer2 (MaxPooling2D)        (None, 128, 128, 64)      0         
_________________________________________________________________
layer3 (Conv2D)              (None, 128, 128, 128)     73856     
_________________________________________________________________
layer4 (Conv2D)              (None, 128, 128, 128)     147584    
_________________________________________________________________
layer5 (MaxPooling2D)        (None, 64, 64, 128)       0         
_________________________________________________________________
layer6 (Conv2D)              (None, 64, 64, 256)       295168    
__________

In [5]:
opt = rmsprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
model_name = 'sketchnet4_4'
%mkdir -p ../logs/{model_name}
%mkdir -p ../models/{model_name}

log_cb4 =\
    TensorBoard(log_dir='../logs/'+model_name+'/', 
                histogram_freq=0, 
                write_graph=False, write_images=False)
best_model_cb4 =\
    ModelCheckpoint('../models/'+model_name+'/best.h5', 
                    monitor='val_acc', verbose=0, 
                    save_best_only=True, 
                    mode='auto', period=1)
latest_model_cb4 =\
    ModelCheckpoint('../models/'+model_name+'/latest.h5', 
                    monitor='val_acc', verbose=0, 
                    period=1)    
def sched_lr(e):
    if e == 0:
        return 0.0001
    elif e < 5:
        return 0.001
    elif e < 10:
        return 0.0005
    elif e < 15:
        return 0.00025
    else:
        return 0.0001
lr_sched_cb4 =\
    keras.callbacks.LearningRateScheduler(sched_lr)

In [7]:
total_epochs = 0

In [ ]:
nb_epoch = 25

for layer in model.layers[:9]:
    layer.trainable = False

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb4, 
                   best_model_cb4,
                   latest_model_cb4,
                   lr_sched_cb4]
)
total_epochs += nb_epoch

Epoch 1/25
265/265 [==============================] - 196s - loss: 0.9027 - acc: 0.7472 - val_loss: 1.2984 - val_acc: 0.7133
Epoch 22/25
265/265 [==============================] - 197s - loss: 0.9049 - acc: 0.7543 - val_loss: 1.2631 - val_acc: 0.7200
Epoch 23/25
265/265 [==============================] - 187s - loss: 0.8571 - acc: 0.7570 - val_loss: 1.3913 - val_acc: 0.6956
Epoch 24/25
265/265 [==============================] - 186s - loss: 0.8887 - acc: 0.7540 - val_loss: 1.2467 - val_acc: 0.7067
Epoch 25/25
265/265 [==============================] - 196s - loss: 0.8744 - acc: 0.7608 - val_loss: 1.2355 - val_acc: 0.7244


In [ ]:
def sched_lr(e):
    if e == 0:
        return 0.0001
    elif e < 10:
        return 0.0005
    elif e < 15:
        return 0.00025
    else:
        return 0.0001
lr_sched_cb4_2 =\
    keras.callbacks.LearningRateScheduler(sched_lr)
    
nb_epoch = 25

for layer in model.layers[:9]:
    layer.trainable = True

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb4, 
                   best_model_cb4,
                   latest_model_cb4,
                   lr_sched_cb4_2]
)
total_epochs += nb_epoch

Epoch 26/50
265/265 [==============================] - 187s - loss: 0.8286 - acc: 0.7702 - val_loss: 1.3786 - val_acc: 0.7052
Epoch 27/50
265/265 [==============================] - 197s - loss: 0.8344 - acc: 0.7706 - val_loss: 1.3898 - val_acc: 0.7267
Epoch 28/50
265/265 [==============================] - 187s - loss: 0.8250 - acc: 0.7860 - val_loss: 1.3067 - val_acc: 0.7215
Epoch 29/50
265/265 [==============================] - 186s - loss: 0.8393 - acc: 0.7774 - val_loss: 1.2297 - val_acc: 0.7156
Epoch 30/50
265/265 [==============================] - 197s - loss: 0.8274 - acc: 0.7777 - val_loss: 1.1853 - val_acc: 0.7311
Epoch 31/50
265/265 [==============================] - 186s - loss: 0.8136 - acc: 0.7691 - val_loss: 1.3304 - val_acc: 0.6948
Epoch 32/50
265/265 [==============================] - 186s - loss: 0.7975 - acc: 0.7751 - val_loss: 1.2965 - val_acc: 0.7200
Epoch 33/50
265/265 [==============================] - 196s - loss: 0.8270 - acc: 0.7728 - val_loss: 1.3608 - val_acc:

In [ ]:
metrics = model.evaluate_generator(validation_generator, steps=135)
print 'val', model.metrics_names, metrics